# Jupyter Notebook Autoencoders

**Model Class:**    Self-Supervised</br>
**Model Type:**     Dimension Reduction Methods</br>

**Purpose**: This notebook provides a template for developing autoencoders. The goal of an autoencoder is to learn a compressed encoding of the input data such that the output from the model matches the input to the model as close as possible. In this example we use the MNIST dataset and create autoencoders to learn encodings that allow the model to output an image of a digit [0-9] that closely resemble the one it received as an input. Various autoencoders have been tested with varying hyperparameters.

**Edit Date** : 10/20/2018

- ##### **Autoencoder models included in this notebook:**
    - Autoencoder as Deep Neural Network (Multi Layer Perceptron)
    - A De-noising (Convolutional) Autoencoder
    - A Variational Autoencoder
</br></br>
- **Resources**
    * https://www.quora.com/What-is-an-auto-encoder-in-machine-learning
    * https://blog.keras.io/building-autoencoders-in-keras.html
    * Deep Learning with Python by Francois Chollet 
    * https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/   
</br>
- **Data**</br>
    - The MNIST dataset is a set of 28x28x1 pixel images of handwritten digits. These are single digits and therefore there are only 10 classes for this dataset [0 - 9].
    - http://yann.lecun.com/exdb/mnist/


- **Goal**</br>
    - Create an autoencoder that can compress the input dimensions into a smaller representation that can be used to reconstruct the image that was fed into the model.
    
# Dependencies

Load the dependencies for model development. Current package requirements include:
* Keras
* Pandas
* Numpy
* Sklearn
* Matplotlib

The current template uses the following versions:
* Keras version: 2.1.5
* Sklearn version: 0.19.1
* Pandas version: 0.22.0
* Numpy version: 1.12.1
* Matplotlib version: 2.0.2

In [ ]:
import keras
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
from keras.models import Sequential
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.utils import np_utils

import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,auc,roc_auc_score,roc_curve
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

from random import shuffle

import numpy as np
import pandas as pd
import matplotlib 
%matplotlib inline
import matplotlib.pyplot as plt

import os
import cv2

In [ ]:
print ("Keras version: " + keras.__version__)
print ("Sklearn version: "+sklearn.__version__)
print ("Pandas version: "+pd.__version__)
print ("Numpy version: "+np.__version__)
print ("Matplotlib version: "+matplotlib.__version__)

# Load Data

### Read the data
* The first step is to import the dataset. Since we are dealing with unstructured data, specifically images, there are several ways to do this. The current notebook uses a dataset which can be imported from the keras libraray itself.
* There is also a function at the end of the notebook that can be used to load data from a local repository.

In [ ]:
# Import the inbuilt Keras MNIST dataset
(X_train, _), (X_test, _) = mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Data Preparation & Transformation

Read in a data set and clean/prepare it for modelling. This activity will heavily depend on your specific problem and the data set that you are using. When possible, a best practice should be to include transformation, scaling, and normalization activities as a function withing your model pipeline. This will simplify the deployment step but is not absolutely necessary.

### Review the Data

In [ ]:
print("Training Data shape: ",X_train.shape)

print("Testing Data shape: ",X_test.shape)

# Build the Models

Keras has two wrappers to use sklearn model pipeline, GridSearchCV : KerasClassifier and KerasRegressor. Since Autoencoders belongs to none of these types, hyper-parameter optimization hasn't been done here.

In [ ]:
# Shape the data to the required dimension
# For typical Deep Neural Network (MLP) the 28by28 image has been flattened. 
# While for Convoluted Neural Network an extra axis is added. 

X_train_MLP = X_train.reshape((X_train.shape[0], -1))
X_test_MLP = X_test.reshape((X_test.shape[0], -1))
print("Training Data_MLP shape: ",X_train_MLP.shape)
print("Testing Data_MLP shape: ",X_test_MLP.shape)

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
print("Training Data shape: ",X_train.shape)
print("Testing Data shape: ",X_test.shape)

##  Autoencoder as Deep Neural Network (Multi Layer Perceptron)

In [ ]:
# Contains 7 hidden layers, 4 as encoder, 3 as decoder.

INPUT_SIZE = 784
HIDDEN_ONE = 512
HIDDEN_TWO = 256
HIDDEN_THREE = 128
ENCODING_SIZE = 64

input_MLP = Input(shape=(INPUT_SIZE,))
MLP_encode = Dense(HIDDEN_ONE, activation='relu')(input_MLP)
MLP_encode = Dense(HIDDEN_TWO, activation='relu')(MLP_encode)
MLP_encode = Dense(HIDDEN_THREE, activation='relu')(MLP_encode)
 
MLP_encode = Dense(ENCODING_SIZE, activation='relu')(MLP_encode)
 
MLP_decode = Dense(HIDDEN_THREE, activation='relu')(MLP_encode)
MLP_decode = Dense(HIDDEN_TWO, activation='relu')(MLP_decode)
MLP_decode = Dense(HIDDEN_ONE, activation='relu')(MLP_decode)
MLP_decode = Dense(INPUT_SIZE, activation='relu')(MLP_decode)
autoencoder_MLP = Model(input_MLP, MLP_decode)

In [ ]:
autoencoder_MLP.compile(optimizer='adam', loss='mean_squared_error' )

autoencoder_MLP.fit(X_train_MLP, 
                    X_train_MLP, 
                    epochs=50, 
                    batch_size=256, 
                    shuffle=True, 
                    validation_split=0.2)

decoded_MLP = autoencoder_MLP.predict(X_test_MLP)

In [ ]:
# Plotting the actual digits and reconstructed digits
plt.figure(figsize=(20, 4))
for i in range(10):
    # original
    plt.subplot(2, 10, i + 1)
    plt.imshow(X_test_MLP[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
    # reconstruction
    plt.subplot(2, 10, i + 1 + 10)
    plt.imshow(decoded_MLP[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
plt.tight_layout()
plt.show()

## A De-noising (Convolutional) Autoencoder

In [ ]:
INPUT_SHAPE = (28, 28, 1)
# Encoder architecture: Input -> Conv2D*5 -> Final encoded image
input_CnAE = Input(shape=(INPUT_SHAPE))
CnAE_encode = layers.Conv2D(32, (3, 3), 
                            activation='relu', 
                            padding='same')(input_CnAE)
CnAE_encode = layers.MaxPooling2D((2, 2), 
                                  padding='same')(CnAE_encode)
CnAE_encode = layers.Conv2D(8, (3, 3), 
                            activation='relu', 
                            padding='same')(CnAE_encode)
CnAE_encode = layers.MaxPooling2D((2, 2), 
                                  padding='same')(CnAE_encode)
CnAE_encode = layers.Conv2D(8, (3, 3), 
                            activation='relu', 
                            padding='same')(CnAE_encode)

CnAE_encode = layers.MaxPooling2D((2, 2), 
                                  padding='same')(CnAE_encode)

# Decoder architecture: Input: Encoded image -> Conv2D-Upsampling*4 -> Conc2D -> Final reconstructed image 
CnAE_decode = layers.Conv2D(8, (3, 3), 
                            activation='relu', 
                            padding='same')(CnAE_encode)
CnAE_decode = layers.UpSampling2D((2, 2))(CnAE_decode)
CnAE_decode = layers.Conv2D(8, (3, 3), 
                            activation='relu', 
                            padding='same')(CnAE_decode)
CnAE_decode = layers.UpSampling2D((2, 2))(CnAE_decode)
CnAE_decode = layers.Conv2D(32, (3, 3), 
                            activation='relu')(CnAE_decode)
CnAE_decode = layers.UpSampling2D((2, 2))(CnAE_decode)
CnAE_decode = layers.Conv2D(1, (3, 3), 
                            activation='relu', 
                            padding='same')(CnAE_decode)

autoencoder_CnAE = Model(input_CnAE, CnAE_decode)

In [ ]:
autoencoder_CnAE.compile(optimizer='adam', loss='mean_squared_error')

autoencoder_CnAE.fit(X_train, 
                     X_train, 
                     epochs=50, 
                     batch_size=64, 
                     shuffle=True, 
                     validation_split=0.2)

decoded_CnAE = autoencoder_CnAE.predict(X_test)

In [ ]:
# Plotting the actual digits and reconstructed digits
plt.figure(figsize=(20, 4))
for i in range(10):
    # original
    plt.subplot(2, 10, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
    # reconstruction
    plt.subplot(2, 10, i + 1 + 10)
    plt.imshow(decoded_CnAE[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
plt.tight_layout()
plt.show()

## A Variational Autoencoder

In [ ]:
img_shape = X_train.shape[1:]
batch_size = 16
latent_dim = 2  # Number of latent dimension parameters (mu:mean , log_sigma:log of standard deviation)

# Encoder architecture: Input -> Conv2D*4 -> Flatten -> Dense
input_VAE = keras.Input(shape=img_shape)

VAE_encode = layers.Conv2D(32, 3, padding='same', activation='relu')(input_VAE)
VAE_encode = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(VAE_encode)
VAE_encode = layers.Conv2D(64, 3, padding='same', activation='relu')(VAE_encode)
VAE_encode = layers.Conv2D(64, 3, padding='same', activation='relu')(VAE_encode)

# need to know the shape of the network here for the decoder
shape_before_flattening = K.int_shape(VAE_encode)

VAE_encode = layers.Flatten()(VAE_encode)
VAE_encode = layers.Dense(32, activation='relu')(VAE_encode)

# Two outputs, latent mean and (log)variance
z_mu = layers.Dense(latent_dim)(VAE_encode)
z_log_sigma = layers.Dense(latent_dim)(VAE_encode)

In [ ]:
# Sampling function : Constructs the image from 'mu' and 'sigma'
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim), mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = layers.Lambda(sampling)([z_mu, z_log_sigma])


In [ ]:
# Decoder takes the latent distribution sample as input
decoder_input = layers.Input(K.int_shape(z)[1:])

# Expand to 784 total pixels
VAE_decode = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)

# Reshape
VAE_decode = layers.Reshape(shape_before_flattening[1:])(VAE_decode)

# Use Conv2DTranspose to reverse the conv layers from the encoder
VAE_decode = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(VAE_decode)
VAE_decode = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(VAE_decode)

# Decoder model statement
VAE_decoder = Model(decoder_input, VAE_decode)

# Apply the decoder to the sample from the latent distribution
VAE_output = VAE_decoder(z)

In [ ]:
# Construct a custom layer to calculate the loss
class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, VAE_decode, VAE_output):
        VAE_decode = K.flatten(VAE_decode)
        VAE_output = K.flatten(VAE_output)
        # Reconstruction loss
        xent_loss = keras.metrics.binary_crossentropy(VAE_decode, VAE_output)
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        return K.mean(xent_loss + kl_loss)

    # Adds the custom loss to the class
    def call(self, inputs):
        VAE_decode = inputs[0]
        VAE_output = inputs[1]
        loss = self.vae_loss(VAE_decode, VAE_output)
        self.add_loss(loss, inputs=inputs)
        return VAE_decode

# Apply the custom loss to the input images and the decoded latent distribution sample
y = CustomVariationalLayer()([input_VAE, VAE_output])


In [ ]:
# VAE model statement
vae = Model(input_VAE, y)


In [ ]:
vae.compile(optimizer='rmsprop', loss=None )

vae.fit(x=X_train, y=None,
        shuffle=True,
        epochs=50,
        batch_size=batch_size,
        validation_data=(X_train, None))

decoded_vae = vae.predict(X_test)

In [ ]:
# Plotting the actual digits and reconstructed digits
plt.figure(figsize=(20, 4))
for i in range(10):
    # original
    plt.subplot(2, 10, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
    # reconstruction
    plt.subplot(2, 10, i + 1 + 10)
    plt.imshow(decoded_vae[i].reshape(28, 28))
    plt.gray()
    plt.axis('off')
 
plt.tight_layout()
plt.show()

<hr/>
# Appendix

## Generic process to read image data from local repository

### Load Data

The first step is to import the dataset. Since we are dealing with unstructured data, specifically images, there are several ways to do this. The current notebook uses a dataset which can be loaded from your local machine.

In [ ]:
# Here the data is stored in three separate folders each with only one image class
# this method will list out all the elements in the directory which makes them easy to import
bird_path = './Data/n01503061'
person_path= './Data/n00007846'
plant_path= './Data/n00017222'

bird_dir = os.listdir(bird_path)
person_dir = os.listdir(person_path)
plant_dir = os.listdir(plant_path)

# Defining the paths for the image
for i in range(len(bird_dir)):
    bird_dir[i] = './Data/n01503061/'+bird_dir[i]
for i in range(len(person_dir)):
    person_dir[i] = './Data/n00007846/'+person_dir[i]
for i in range(len(plant_dir)):
    plant_dir[i] = './Data/n00017222/'+plant_dir[i]

# Data Preparation & Transformation

Read in a data set and clean/prepare it for modelling. This activity will heavily depend on your specific problem and the data set that you are using. When possible, a best practice should be to include transformation, scaling, and normalization activities as a function withing your model pipeline. This will simplify the deployment step but is not absolutely necessary.


In [ ]:
# Creating class labels for the data
bird_name = []
person_name = []
plant_name = []
for i in range(len(bird_dir)):
    bird_name.append('Bird')
for i in range(len(person_dir)):
    person_name.append('Person')
for i in range(len(plant_dir)):
    plant_name.append('Plant')

In [ ]:
# This will group the data with their label in a zip data structure
birds = list(zip(bird_dir, bird_name))
people = list(zip(person_dir,person_name))
plants = list(zip(plant_dir,plant_name))

data = birds+ people+ plants

In [ ]:
# This randomly shuffles the data for better sampling 
shuffle(data)
addrs, labels = zip(*data)

In [ ]:
labels = np.array(labels)

### Loading the data and preprocessing

In [ ]:
dataset = []
i = 0
# This method will help you load data as numpy arrays
for image_paths in addrs:  
    image = cv2.imread(image_paths)
    image = cv2.resize(image, (224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    dataset.append(image)
    

In [ ]:
# Format the data
dataset = np.array(dataset, dtype = 'int32')
dataset.shape

In [ ]:
# Lets take alook at the images:
image = cv2.imread(data[3900][0])
# OpenCV reads images as BGR channels. In order to view images properly, we canuse cvtColor function to revert to regular RGB channels
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
plt.imshow(image)
print(labels[3900])

In [ ]:
# Label encoding
lb = LabelEncoder()
labels = lb.fit_transform(labels)

In [ ]:
np.unique(labels)
nb_classes = 3

# Create Training and Validation Data Sets
Create the training and test splits that will be used to train and evaluate the model. Some considerations that may impact model performance when creating the train test splits include:
* Class balance - Are the classes well balanced or should the data set be resampled prior to model development
* Split Size - How large are the training and test splits. This will depend on the sample size and training strategy including using a cross validation
* Random Seed - For reproducable experiments, a consistent random seed should be used

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, random_state = 42, test_size = 0.2)

In [ ]:
#normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Y_train = np_utils.to_categorical(y_train,3)
Y_test = np_utils.to_categorical(y_test,3)

In [ ]:
print("Training Data shape: ",x_train.shape)
print("Testing Data shape: ",x_test.shape)